<h1> 2. Creating a sampled dataset </h1>

This notebook illustrates:
<ol>
<li> Sampling a BigQuery dataset to create datasets for ML
<li> Preprocessing with Pandas
</ol>

In [1]:
# change these to try this notebook out
BUCKET = 'qwiklabs-gcp-c013b2c99462e171'
PROJECT = 'qwiklabs-gcp-c013b2c99462e171'
REGION = 'us-east1-b'

In [5]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [6]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

<h2> Create ML dataset by sampling using BigQuery </h2>
<p>
Let's sample the BigQuery data to create smaller datasets.
</p>

## Lab Task #1

Sample the BigQuery resultset (above) so that you have approximately 12,000 training examples and 3000 evaluation examples.
The training and evaluation datasets have to be well-distributed (not all the babies are born in Jan 2005, for example)
and should not overlap (no baby is part of both training and evaluation datasets).

Hint (highlight to see): <p style='color:white'>You will use MOD() on the hashmonth to divide the dataset into non-overlapping training and evaluation datasets, and RAND() to sample these to the desired size.</p>

In [7]:
# Create SQL query using natality data after the year 2000
import google.datalab.bigquery as bq
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
"""

In [18]:
sq = "SELECT COUNT(weight_pounds) FROM (" + query + ") WHERE MOD(ABS(hashmonth),10)<8 AND RAND() < 0.0004"

In [19]:
df = bq.Query(sq).execute().result().to_dataframe()

In [21]:
df

,f0_
0,10876


## Lab Task #2

Use Pandas to:
* Clean up the data to remove rows that are missing any of the fields.
* Simulate the lack of ultrasound.
* Change the plurality column to be a string.

Hint (highlight to see): <p>
Filtering:
<pre style='color:white'>
df = df[df.weight_pounds > 0]
</pre>
Lack of ultrasound:
<pre style='color:white'>
nous = df.copy(deep=True)
nous['is_male'] = 'Unknown'
</pre>
Modify plurality to be a string:
<pre style='color:white'>
twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
df['plurality'].replace(twins_etc, inplace=True)
</pre>
</p>

In [41]:
sq2 = """
SELECT * FROM(
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  ABS(FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING)))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000 AND is_male IS NOT null AND plurality IS NOT null AND weight_pounds IS NOT null AND gestation_weeks IS NOT null)
WHERE MOD(ABS(hashmonth),10)<8 AND RAND() < 0.0004
"""

In [42]:
df = bq.Query(sq2).execute().result().to_dataframe()

In [32]:
df.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,6.812284,False,13,1,40.0,1891060869255459203
1,6.563162,True,13,1,41.0,7108882242435606404
2,8.000575,False,14,1,38.0,1866590652208008467
3,6.563162,False,14,1,40.0,5960947298512945432
4,8.624484,False,14,1,40.0,1525201076796226340


In [43]:
df.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hashmonth
count,10596.000000,10596.000000,10596.000000,10596.000000,1.059600e+04
mean,7.206021,27.392223,1.033409,38.589185,4.426500e+18
std,1.338438,6.145936,0.195314,2.589461,2.814294e+18
min,0.639341,13.000000,1.000000,20.000000,7.493147e+16
25%,6.563162,23.000000,1.000000,38.000000,1.639186e+18
50%,7.312733,27.000000,1.000000,39.000000,4.701948e+18
75%,8.000575,32.000000,1.000000,40.000000,6.910175e+18
max,13.000660,50.000000,5.000000,47.000000,9.183606e+18


In [45]:
import copy

In [46]:
df2 = copy.deepcopy(df)


In [52]:
df2['is_male'] = 'Unknown'
df2['plurality'].loc([df2['plurality'] == 1]) = 'Single'
df2.head()

SyntaxError: can't assign to function call (<ipython-input-52-2760f6b12bc0>, line 2)

## Lab Task #3

Write the cleaned out data into CSV files.  Change the name of the Pandas dataframes (traindf, evaldf) appropriately.



In [ ]:
traindf.to_csv('train.csv', index=False, header=False)
evaldf.to_csv('eval.csv', index=False, header=False)

In [ ]:
%bash
wc -l *.csv
head *.csv
tail *.csv

Copyright 2017-2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License